## Import Libraries

In [73]:
import pandas as pd
import pickle as pkl

## Read in Dataframes / Pickle Files

In [82]:
with open('df_subset.pkl', 'rb') as f:
    df_subset = pkl.load(f) # deserialize using load()

with open('apriori-rules.pkl', 'rb') as f:
    apriori_rules = pkl.load(f) # deserialize using load()

with open('cosine-similarities.pkl', 'rb') as f:
    cosine_sim3 = pkl.load(f) # deserialize using load()

with open('top_30_cosine_similarity.pkl', 'rb') as f:
    cosine_sim_30 = pkl.load(f) # deserialize using load()

In [146]:


df_subset[df_subset['id'] == 492410.0]['Unnamed: 0'].values[0]


5856

In [75]:
# list of all recipes that have a corresponding relation to another recipe (collaborative-filtering)
recipe_rules = list(apriori_rules['Base Product'].unique())
print(recipe_rules)

[486261, 495124, 495275, 495577, 494631, 514965, 494435, 487073, 494972, 494609, 494784, 495271, 494671, 494303, 495152, 487492, 487593, 495967, 487551, 496573, 495134, 487039, 514423, 487669, 493958, 518145, 518069, 496591, 475041, 496730, 518068, 518151, 493963, 496552, 497295, 497130, 497382, 497236, 497261, 518143, 487568, 494953, 475780, 475819, 518229, 496767]


In [76]:
def recommendation_engine(user_rated_recipes):
    for k, v in user_rated_recipes.items(): 
        if not v < 3:
            print(k, '>', v)

In [77]:
#dummy data
user_rated_recipes = {5856:2, 5991:4, 2004:5}

recommendation_engine(user_rated_recipes)

5991 > 4
2004 > 5


In [174]:
apriori_rules

,Base Product,Add Product,Support
0,486261,"(486267,)",0.004377
1,495124,"(495577,)",0.003307
2,495275,"(495577,)",0.002918
3,495124,"(495275,)",0.002724
4,495577,"(496573,)",0.002724
...,...,...,...
1510,496767,"(495275,)",0.001070
1511,494303,"(496552, 494435)",0.001070
1512,497295,"(495275,)",0.001070
1513,487492,"(487669, 494303)",0.001070


In [173]:
apriori_rules[apriori_rules['Base Product'] == 486261]['Add Product']

0    (486267,)
Name: Add Product, dtype: object

## Recommendation Engine

In [170]:
def get_recommendations(recipe_index=486261):

    # Fill first with apriori rules (collaborative filtering), then cosine sim rules (content filtering)
    remaining = 30 - min(30,len(apriori_rules[apriori_rules['Base Product'] == recipe_index]))

    recipe_apriori_df = pd.DataFrame(columns = [['index', 'name', 'score']])

    if remaining < 30:

        # INTEGRATE APRIORI RULES
        
        # for i in range(0, 30-remaining):
        #     print(i)

        #     df_id = df_subset[df_subset['id'] == recipe_id]['Unnamed: 0'].values[0]

        #     recipe_index = 
        #     recipe_name = 
        #     recipe_score = 
            
      
        
        

    # Select columns containing ids and scores
    id_columns = [col for col in cosine_sim_30.columns if col.startswith('id_')]
    score_columns = [col for col in cosine_sim_30.columns if col.startswith('score_')]

    # list to contain pairs of ids and similarity scores
    output = []

    # iterate through ids and scores together
    for id_col, score_col in zip(id_columns, score_columns):

        # recipe ids
        recipe_id = cosine_sim_30[cosine_sim_30['id'] == recipe_index][id_col].values[0]

        # get index in dataframe from recipe id
        df_id = df_subset[df_subset['id'] == recipe_id]['Unnamed: 0'].values[0]

        # recipe similarity score
        recipe_score =cosine_sim_30[cosine_sim_30['id'] == recipe_index][score_col].values[0]

        # append the pair of df index and rceipe score to list
        output.append([df_id, recipe_score])


    # Get the scores of the remaining (0-30) most similar recipes
    output = output[0:remaining]

    # Get the recipe indices and corresponding recipe similarity score
    recipe_indices = [i[0] for i in output]
    recipe_similarity_scores = [i[1] for i in output]

    recommendations_df = df_subset['name'].iloc[recipe_indices].to_frame().reset_index()
    recommendations_df['score'] = recipe_similarity_scores

# Return the top 10 most similar recipes
    return recommendations_df

In [171]:
get_recommendations()  

29


,index,name,score
0,6858,taco nachos rsc,0.412908
1,6963,cheesy roasted corn red pepper black bean di...,0.398191
2,1421,baked mexican pie,0.394854
3,7109,mexican fish packets rsc,0.389094
4,6752,ranchero jack veggie quiche rsc,0.378074
5,6698,vegetarian tex mex tamales rsc,0.373667
6,7638,beef tortilla stack,0.373503
7,6839,hobo adobo foil dinner rsc,0.367710
8,6311,overnight mexican tortilla lasagna,0.359453
9,6694,tex mex beef bean tamales rsc,0.358548
